# Convolutional Neural Networks

Convolutional Neural Networks are very similar to ordinary Neural Networks from the previous chapter: they are made up of neurons that have learnable weights and biases. Each neuron recieves some inout, performs a dot product and optionally follows it with a non-linearity. The whole network still expresses a single differentiable score function: from raw images pixels on one end to class scores at the other. And they still have a loss function on the last full connected layer and all the tips/ tricks we developed for learning regular Neural Networks still apply. So what does change? ConvNet architectures make the explicit assumption that the inputs are images, which allows us to encode certain properties into the architecture. These then make the forward function more efficient to implement and vastly reduce the amount of parameters in the network.

*Recall: Regular Neural Nets*: As we saw in the previous chapter, Neural Networks recieve an input (a single vector) and transform it through a series of _hidden layers._ Each hidden layer is made up of a set of neurons, where each neuron is fully connected to all neurons in the previous layer, and where neurons in the single layer function completely independently and do not share any connections. The last fully connected layer is called the output layer and in classification settings it represents the class score.

 _Regular Neural Nets don't scale well to full images._ In CIFAR-10, images are only size 32x32x3 (32 wide, 32 high, 3 color channels), so a single fully connected neuron in a first hidden layer of a regular Neural Network would have 32*32*3 = 3072 weights. This amount still seems manageable, but clearly this fully connected structure does not scale to larger images. For example, an image of more respectable size e.g. 200x200x3 would lead to neurons that have 200*200*3 = 120,000 weights. Moreover, we would almost certainly want to have several such neurons so the parameters would add up quickly! Clearly, this full connectivity is wasteful and the huge number of parameters would lead to overfitting.

_3D volume of neurons_. Convolutional Neural Networks take advantage of the fact that the input consists of images and they constrain the architecture in a more sensible way. In particular, unlike a regular Neural Network, the layers of a ConvNet have neurons arranged in 3 dimensions: **width,height, depth** (Note that the word depth here refers to the third dimension of an activation volume, not to the depth of a full Neural Network, which can refer to the total number of layers in a network). For example, the input images in CIFAR-10 are an input volume of activations, and the volume has dimensions 32x32x3 (width,height, depth). As we will soon see, the neurons in a layer will only be connected to a small region of the layer before, instead of all the neurons in a full-connected manner. Moreover, the final output layer would for CIFAR-10 have a dimensions 1x1x10, because by the end of the ConvNet architecture we will reduce the full image into a single vector of class scores, arranged along the depth dimension. Here is a visualization:

![](http://cs231n.github.io/assets/cnn/cnn.jpeg)

* Left: A regular 3-layer Neural Network. Right: A ConvNet arranges its neurons in three dimensions (width, height, depth), as visualized in one of the layers. Every layer of a ConvNet transforms the 3D input volume to a 3D output volume of neuron activations. In this example, the red input layer holds the image, so its width and height would be the dimensions of the image, and the depth would be 3 (Red, Green, Blue channels).

#### A ConvNet is made up of Layers. Every Layer has a simple API: It transforms an input 3D volume to an output 3D volume with some differentiable function that may or may not have parameters.

___

## Layers used to build ConvNets

As we described above, a simple ConvNet is a sequence of layers, and every layer of the Convnet transforms one volume of activation to another through differentiable functions. We use three main types of layers to build ConvNet architectures: **Convolutional Layer, Pooling Layer** and **Fully-Connected Layer** (exactly as seen in regular Neural Networks). We will stack these layers to form a full ConvNet **architecture**.

*Example Architecture Overview.* We will go into more depth below, but a simple ConvNet for CIFAR-10 classification could have the architecture [INPUT -CONV - RELU - POOL -FC]. In more detail:

* INPUT [32x32x3] will hold the raw pixel values of the image, in this case an image of width 32, height 32, and with three color channel R,G,B.

* CONV layer will compute the output of neurons that are connected to local regions in the input, each computing a dot product between their weights and a small region they are connected to in the input volume. This may result in volume such as [32x32x12], if we decided to use 12 filters.

* RELU layer will apply an elementwise activation function, such as the **max(0,x)**thresholding at zero. This leaves the size of the volume unchanged([32 x 32 x 12]) 

* POOL layer will perform a downsampling operation along the spatial dimensions(width, height), resultin in volume such as [16x16x12].

* FC (i.e. fully-connected) layer will compute the class scores, resulting in volume of size [1x1x10], where each of the 10 numbers correspond to a class score, such as among the 10 categories of CIFAR-10. As with ordinary Neural Networks and as the name implies, each neuron in this layer will be connected to all the numbers in the previous volume.

In this way, ConvNets transform the original image layer by layer from the original pixel values to the final class scores. Note that some layers contain parameters and other don't. In particular, the CONV/FC layers perform transformations that are a function of not only the activations in the input volume, but also of the parameters (the weights and biases of the neurons). On the other hand, RELU/POOL layers will implement a fixed function. The parameters in the CONV/FC layers will be trained with gradient descent so that the class scores that the ConvNet computes are consistent with the labels in the training set for each image.


In summary 
* A ConvNet architecture is in the simpliest case a list of Layers that transform the image volume into an output volume (e.g. holding the class score). 

* There are a few distinct types of Layers (e.g. CONV/FC/RELU/POOL are by far the most popular).

* Each Layer accepts an input 3D volume and transforms it to an output 3D volume through a differentiable function.

* Each Layer may or may not have parameters

* Each layer may or may not have additional hyperparameters.

![](http://cs231n.github.io/assets/cnn/convnet.jpeg)

* The activations of an example ConvNet architecture. The initial volume stores the raw image pixels (left) and the last volume stores the class scores (right). Each volume of activations along the processing path is shown as a column. Since it's difficult to visualize 3D volumes, we lay out each volume's slices in rows. The last layer volume holds the scores for each class, but here we only visualize the sorted top 5 scores, and print the labels of each one. The full web-based demo is shown in the header of our website. The architecture shown here is a tiny VGG Net, which we will discuss later.

## Convolutional Layer

The conv layer is the core building block of a Convolutional Network that does most of the computational heavy lifting.

____


**Overview and Intuition without brain stuff.** Lets first discuss what the CONV layer computes without brain/neuron analogies. The CONV layer's parameters consist of a set of learnable filters. Every filter is small spatially (along width and height), but extends through the full depth of the input volume. For example, a typical filter on a first layer of a ConvNet might have size 5x5x3 (i.e. 5 pixels width and height and 3 because images have depth 3, the color channels). During the forward pass, we slide (more precisely **convolve**) each filter across the width and height of the input volume and compute the dot product between the entries of the filter and the input at any position. As we slide the filter over the width and height of the input volume we will produce a 2-dimensional activation map that gives the response of that filter at every spatial position. Intuitively, the network will learn fitlers that activate when they see some type of visual feature such as an edge of some orientation or a blotch of some color on the first layer, or eventually entire honeycomb or wheel-like patterns on higher layers of the network. Now we have an entire set of filters in each CONV layer (e.g. 12 filers), and each of them will produce a seperate 2-dimensional activation map. We will stack these activation maps along the depth dimension and produce the output volume.

**Local Connectivity**: When dealing with high-dimensional inputs such as images, as we saw above it is impractical to connect neurons to all neurons in the previous volume. Instead, we will connect each neuron to only a local region of the input volume. The spatial extent of this connectivity is a hyperparameter called the **receptive field** of the neuron(filter size). The extent of the connectivity along the depth axis is always equal to the depth of the input volume. It is important to emphasize again this asymmetry in how we treat the spatial dimensions (width and height) and the depth dimension: The connections are local in space (along width and height), but always full along the entire depth of the input volume

*Example 1.* For example, suppose that the input volume has size [32x32x3] (eg. an RGB CIFAR 10 image). if the receptive field (or the filter size) is 5x5 then each neuron in the Conv Layer will have weights to the [5x5x3] region of the input volume, for a total of 5 x 5 x 3 = 75 weights(and +1 bias parameter). Notice that the extent of the connectivity along the depth axis must be 3, since this is the depth of the input volume.

![](http://cs231n.github.io/assets/cnn/depthcol.jpeg)

![](http://cs231n.github.io/assets/nn1/neuron_model.jpeg)

* Left: An example input volume in red (e.g. a 32x32x3 CIFAR-10 image), and an example volume of neurons in the first Convolutional layer. Each neuron in the convolutional layer is connected only to a local region in the input volume spatially, but to the full depth (i.e. all color channels). Note, there are multiple neurons (5 in this example) along the depth, all looking at the same region in the input - see discussion of depth columns in text below. Right: The neurons from the Neural Network chapter remain unchanged: They still compute a dot product of their weights with the input followed by a non-linearity, but their connectivity is now restricted to be local spatially.

____

** Spatial arrangement **. We have explained the connectivity of each neuron in the Conv Layer to the inout volume, but we haven't yet discussed how many neurons there are in the output volume or how they are arranged. Three hyperparameters control the size of the output volume: the **depth, stride** and **zero-padding**. We discuss these next:

1. Firstly, the **depth** of the output volume is a hyperparameter: it corresponds to the number of filters we would like to use, each learning to look for something different in the input. For example, if the first Convolutional Layer takes as input the raw image, then different neurons along the depth dimension may activate in presence of various oriented edges, or blobs of color. We will refer to a set of neurons that are all looking at the same region of the input as a **depth column**.


2. Second, we must specify the **stride** with which we slide the filter. When the stride is 1 then we move the filter one pixel at a time. When the stride is 2 then the filters jump 2 pixels at a time as we slide them around. This will produce a smaller output volume spatially.


3. As we will soon see, sometimes it will be convenient to pad the input volume with zeros around the boarder. The size of this **zero-padding** is a hyperparameter. The nice feature of zero padding is that it will allow us to control the spatial size of the output volume.

We can compute the spatial size of the output volume as a function of the input volume size ($W$), the receptive field size of the Conv Layer neurons ($F$), the stride with which they are applied ($S$), and the amound of zero padding used ($P$) on the border. You can convince yourself that the correct formula for calculating how many neurons *fit*, is given by ($W-F+2P/S+1 $). FOr example, for a 7 x 7 input and a 3 x 3 filter with stride 1 and a padding of 0, we would get a 5 x 5 output. WIth stride 2 we would get a 3 x 3 output. 

_Use of zero-padding._ In the example above on left, note that the input dimension was 5 and the output dimension was equal: also 5. This works out because our receptive fields were 3 and we used zero padding of 1. If there was no zero padding used, then the output volume would have had spatial dimension of only, because that is how many neurons would have "fit" across the original input. In general, setting zero padding to be $ P = (F-1)/2$ when the stride is $ S = $ ensures that the input volume and output volume will have the same size spatially. It is very common to use zero-padding in this way and we will discuss the full reasons when we talk more about ConvNet architectures.

Constraints on strides. Note again that the spatial arrangement hyperparameters have mutual constraints. For example, when the input has size W=10W=10, no zero-padding is used P=0P=0, and the filter size is F=3F=3, then it would be impossible to use stride S=2S=2, since (W−F+2P)/S+1=(10−3+0)/2+1=4.5(W−F+2P)/S+1=(10−3+0)/2+1=4.5, i.e. not an integer, indicating that the neurons don’t “fit” neatly and symmetrically across the input. Therefore, this setting of the hyperparameters is considered to be invalid, and a ConvNet library could throw an exception or zero pad the rest to make it fit, or crop the input to make it fit, or something. As we will see in the ConvNet architectures section, sizing the ConvNets appropriately so that all the dimensions “work out” can be a real headache, which the use of zero-padding and some design guidelines will significantly alleviate.